In [7]:
import torch
import time
!pip install triton==2.2.0
!pip install validators

  Using cached triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (167.9 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 781.2 kB/s eta 0:00:00


In [8]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Using cpu for inference


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [9]:
# Load model to the gpu
load_start_time = time.time()
resnet50.eval().to(device)
if device.type == 'cuda':
    torch.cuda.synchronize() # Make sure all operations are finished
load_end_time = time.time()

In [10]:
# Load samples to the gpu
uris = [
    'http://images.cocodataset.org/test-stuff2017/000000024309.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000028117.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000006149.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000004954.jpg',
]
batch = torch.cat(
    [utils.prepare_input_from_uri(uri) for uri in uris]
).to(device)

In [14]:
# Start inferencing
total_samples = 0
inference_start_time = time.time()
with torch.no_grad():
    output = torch.nn.functional.softmax(resnet50(batch), dim=1)
    total_samples += batch.size(0)
if device.type == 'cuda':
    torch.cuda.synchronize() # Make sure all operations are finished
inference_stop_time = time.time()

In [15]:
# Getting the result
inference_time = inference_stop_time - inference_start_time
throughput = total_samples / inference_time

print(f"Total samples : {total_samples}")
print(f"Throughput: {throughput:.2f} samples/second")
print(f"Load model latency: {load_end_time - load_start_time}")
print(f"Inference latency: {inference_time}")

results = utils.pick_n_best(predictions=output, n=5)
print(results)

Total samples : 4
Throughput: 4.67 samples/second
Load model latency: 0.03139019012451172
Inference latency: 0.8567917346954346
sample 0: [('laptop, laptop computer', '34.7%'), ('mouse, computer mouse', '11.9%'), ('notebook, notebook computer', '10.9%'), ('monitor', '3.9%'), ('web site, website, internet site, site', '3.1%')]
sample 1: [('mashed potato', '78.6%'), ('broccoli', '7.3%'), ('meat loaf, meatloaf', '2.5%'), ('plate', '1.8%'), ('guacamole', '0.2%')]
sample 2: [('racket, racquet', '25.4%'), ('tennis ball', '6.0%'), ('ping-pong ball', '2.0%'), ('catamaran', '0.3%'), ('bathtub, bathing tub, bath, tub', '0.3%')]
sample 3: [('groenendael', '12.7%'), ('Scottish deerhound, deerhound', '11.2%'), ('flat-coated retriever', '6.6%'), ('kelpie', '6.1%'), ('Great Dane', '3.1%')]
[[('laptop, laptop computer', '34.7%'), ('mouse, computer mouse', '11.9%'), ('notebook, notebook computer', '10.9%'), ('monitor', '3.9%'), ('web site, website, internet site, site', '3.1%')], [('mashed potato', '78